In [1]:
from collections import Counter 
  
import os
import time as time
import ezdxf

from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.cluster.hierarchy import fcluster


In [112]:
from collections import Counter 
  
import os
import time as time
import ezdxf
import numpy as np
from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.cluster.hierarchy import fcluster




## Version 1 --> 
## algorithm: 
## find all vertices (areas are discarded) in alignment layer
## group vertices based on distance, separation between marks > max_d > single alignment mark
## detect center of mark, create pairwise average points, the center should have the most pairwise points having it as center
## requirements: mark mainly symmetric around
##               center of mark 


folder = '/Users/Lstaqdev/Downloads/Testalignmentmarks/' #folder for dxf in and dat output

file = 'SAGHUBBARDreduced.dxf' #name of DXF

max_distance = 213 #max size of alignment mark in µm

#layers=['ebeam','outerebam'] #Layer(s) containining alingment marks


paths=os.listdir()   #or could be done with a folder but i think its best to search in the folder the script is sitting in
                        #so ideally there would be only one file in that folder, then program can autofind it

filename=finddxforGdsii(paths)

#filename=folder+file

doc = ezdxf.readfile(filename)

layer = finddxflayer(doc,keyword='')

marks=findallalignmentmarks(filename,layer,max_d=213)


Multiple design files. Type # of file and confirm with enter 

#1 ABDesign310518_FNTJSWv3.dxf
#2 Clewin HUBBARD FULL horizontal.dxf
#3 clewin4thewinsag.dxf
#4 DesignArrays.dxf
#5 fulldesign_vol3.dxf
#6 hubbardtest.dxf
#7 it werks.dxf
#8 itwerks.dxf
6
Multiple layers. Type # of layer and confirm with enter 

#1 0
#2 LAYER_1
#3 LAYER_2
#4 LAYER_3
#5 LAYER_4
#6 LAYER_5
#7 LAYER_6
#8 Defpoints
5
object found
object found
object found
object found
object found
object found
object found
object found
object found
object found
object found
object found
object found
object found
object found
object found
object found
object found
object found
object found
object found
object found
object found
object found
object found
object found
object found
object found
object found
object found
object found
object found
object found
object found
object found
object found
object found
object found
object found
object found
object found
object found
object found
object found
object found
object found
object 

TypeError: '_GeneratorContextManager' object is not iterable

In [43]:
foldercontent = os.listdir()

In [92]:
msp = doc.modelspace()
    
points=[]
    
polylines = msp.query()
for polyline in polylines:
        if polyline.dxf.layer in layers:
            for i, location in enumerate(polyline.points()):
                print('Point at index {}: {}'.format(i, location))
                if [location[0],location[1]] not in points:
                    points.append([location[0],location[1]])
                    

In [52]:
rndmarks=[(75.0, 75.0), (75.0, 4925.0), (4925.0, 75.0), (4925.0, 4925.0)]

In [53]:
rndmarks[0][0]

75.0

In [29]:
sconcontent=open('12_mesa_10nA_A4_.scon','r')

In [30]:
sconcontent.read()

'/*12_mesa_10nA_A4_.scon*/\n/* BEAMER CREATED = Wed Jan 15 15:31:26 2020 */\n/* BEAMER BASEDOSE = 0.050000 */\n/* BEAMER EXTENT = 848.021500, 853.597000, 3248.021500, 3255.619000 */\nVER, 1.120615\nMODEL, F125-50B (Subfield)\nID, unknown\nMF, 250.000\nFLD, \n!START\nSP, 5.000,5.000\nSZ, 250\nPX, 50000\nRL4, 1.844, 2.673, 2.236, 2.673, 1.844, 2.827, 2.236, 2.827\nPC, 12_mesa_10n00000, 1.8280200000, 2.5385950000\nPC, 12_mesa_10n00001, 2.0780200000, 2.5385950000\nPC, 12_mesa_10n00002, 2.2905200000, 2.5385950000\nPC, 12_mesa_10n00003, 2.2905200000, 2.7885950000\nPC, 12_mesa_10n00004, 2.0780200000, 2.7885950000\nPC, 12_mesa_10n00005, 1.8280200000, 2.7885950000\nPC, 12_mesa_10n00006, 1.8280200000, 3.0010950000\nPC, 12_mesa_10n00007, 2.0780200000, 3.0010950000\nPC, 12_mesa_10n00008, 2.2905200000, 3.0010950000\n!END'

In [68]:
rndscon='11_mesa_10nA_A4_NM.scon'

In [69]:
writemarksintoscon(rndscon,rndmarks)

In [67]:
def writemarksintoscon(scon,marks):
    f=open(scon,'r')
    scontent=f.read()
    if any(regtype in scontent for regtype in ['RL2 ','RL3 ','RL4 ']):
        print('File already has marks, not overwriting')
        return
    sconparts=scontent.split('PC, ')
    if len(marks)==4:
        outputscon=sconparts[0]+'RL4 %.3f, %.3f, %.3f, %.3f, %.3f, %.3f, %.3f, %.3f \n' %(marks[0][0],marks[0][1],marks[1][0],marks[1][1],marks[2][0],marks[2][1],marks[3][0],marks[3][1])
    elif len(marks)==3:
        outputscon=sconparts[0]+'RL3 %.3f, %.3f, %.3f, %.3f, %.3f, %.3f, %.3f, %.3f \n' %(marks[0][0],marks[0][1],marks[1][0],marks[1][1],marks[2][0],marks[2][1])    
    elif len(marks)==2:
        outputscon=sconparts[0]+'RL2 %.3f, %.3f, %.3f, %.3f, %.3f, %.3f, %.3f, %.3f \n' %(marks[0][0],marks[0][1],marks[1][0],marks[1][1]) 
    for index,i in enumerate(sconparts):
        if index>0:
            outputscon=outputscon+'PC, '+i
        
    outputfilename=scon[:-5]+'Marks'+scon[-5:]
    
    f= open(outputfilename,"w+")
    f.write(outputscon)
    f.close
    
    
        
    

In [34]:
def findscon(paths):
    allscon=[]
    userindex=1
    for file in paths:
        if '.scon' in file:
            allscon.append(file)
    if len(allscon)==0:
        print('No Scon file in folder')
        return
    elif len(allscon)>1:
        print('Multiple Scon files. Type # of file and confirm with enter \n')
        for index,sconfile in enumerate(allscon):
                print('#'+str(index+1)+' '+sconfile)
        userindex=int(input())
    return(allscon[userindex-1])

In [71]:
def finddxforGdsii(paths):
    allvectorgraphic=[]
    userindex=1
    for file in paths:
        if '.dxf' in file or '.gds' in file:
            allvectorgraphic.append(file)
    if len(allvectorgraphic)==0:
        print('No design file in folder')
        return
    elif len(allvectorgraphic)>1:
        print('Multiple design files. Type # of file and confirm with enter \n')
        for index,vecfile in enumerate(allvectorgraphic):
                print('#'+str(index+1)+' '+vecfile)
        userindex=int(input())
    return(allvectorgraphic[userindex-1])

In [47]:
for layer in doc.layers:
    print(layer.dxf.name)

0
Arms_General_Purpose
EBEAM1
XX2
XX5
XX1
guidelines
*ADSK_ASSOC_ENTITY_BACKUPS
XX4
*ADSK_CONSTRAINTS
XX3
SAG_Teaching_Devices
SAG_ARRAY
Text1
Arms_Ohmics
Arms_Sandwich_Gate
Arms_Top_Gate
LAYER_1
LAYER_2
ebeam
outerebam
Defpoints


In [110]:
def finddxflayer(doc,keyword=''):
    alllayers=[]
    for layer in doc.layers:
        if keyword.lower() in layer.dxf.name.lower():
            alllayers.append(layer.dxf.name)
    
    userindex=1
   
    if len(alllayers)==0:
        print('No layer with keyword found in folder')
        return
    elif len(alllayers)>1:
        print('Multiple layers. Type # of layer and confirm with enter \n')
        for index,layer in enumerate(alllayers):
                print('#'+str(index+1)+' '+layer)
        userindex=int(input())
    return([alllayers[userindex-1]])

In [5]:
finddxflayer(doc,keyword='SAg')

Multiple layers. Type # of layer and confirm with enter 

#1 SAG_Teaching_Devices
#2 SAG_ARRAY
2


'SAG_ARRAY'

In [8]:
def findoutermostmarks(marks):   # and sort them
    LL=[0,-1000000]
    LR=[0,-1000000]
    UR=[0,-1000000]
    UL=[0,-1000000]
    for markindex,mark in enumerate(marks):
        LLscore=-mark[0]-mark[1]
        LRscore=-mark[0]+mark[1]
        ULscore=mark[0]-mark[1]
        URscore=mark[0]+mark[1]
        if LLscore>LL[1]:
            LL[1]=LLscore
            LL[0]=markindex
        if LRscore>LR[1]:
            LR[1]=LRscore
            LR[0]=markindex
        if ULscore>UL[1]:
            UL[1]=ULscore
            UL[0]=markindex
        if URscore>UR[1]:
            UR[1]=URscore
            UR[0]=markindex
    return [marks[LL[0]],marks[LR[0]],marks[UL[0]],marks[UR[0]]]

In [16]:
findoutermostmarks(alignmentmarks)

[(75.0, 75.0), (75.0, 4925.0), (4925.0, 75.0), (4925.0, 4925.0)]

In [107]:
msp = doc.modelspace()
for e in msp:
    print(e.dxftype())
# entity query for all LINE entities in modelspace
for e in msp.query('LINE'):
    print_entity(e)

LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE

LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE

LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE

LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE

LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE

...

LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE

LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE

LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE

LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE

LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE
LWPOLYLINE

In [15]:
alignmentmarks=findallalignmentmarks(doc,layers)

In [108]:
def findallalignmentmarks(filename,layers,max_d=220):
    
    doc = ezdxf.readfile(filename)
    
    msp = doc.modelspace()
    
    points=[]
    
    polylines = msp.query('LWPOLYLINE') #or 'POLYLINE' need to make it more consistent
    
    
    for polyline in polylines:
        print('object found')
        if polyline.dxf.layer in layers:
            print('rightlayer')
            for i, location in enumerate(polyline.points()):
                print('Point at index {}: {}'.format(i, location))
                if [location[0],location[1]] not in points:
                    points.append([location[0],location[1]])
      
    print(points)
        
    Z = linkage(points,method='complete',metric='euclidean')                           
    
    
                     
    clusters = fcluster(Z, max_d, criterion='distance')
    
    
    allclusters=[]
    for i in range(1,max(clusters)+1):
        clusterslist=[]
        for j in range(len(clusters)):
            
            if clusters[j]==i:
                clusterslist.append(points[j])
        allclusters.append(clusterslist)
    
        
        
     
    
    
    alignmentmarks = []
    for j in allclusters:
        centers=[]
        for i in j:
            for k in j:
                centers.append(((i[0]+k[0])/2,(i[1]+k[1])/2))
        
        alignmentmarks.append(most_frequent(centers))
    
    return alignmentmarks


def most_frequent(List): 
    occurence_count = Counter(List) 
    return occurence_count.most_common(1)[0][0] 
    